In [9]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv('train.csv',sep='\t')

# 创建一个列表，包含允许的标签
allowed_labels = ['童书', '工业技术', '大中专教材教辅']

# 根据条件筛选行
df = df[df['label'].isin(allowed_labels)]

# 保存筛选后的结果到新的CSV文件
df.to_csv('train2.csv', index=False)

TypeError: 'DataFrame' object is not callable

In [10]:
df

,title,desc,text,label
0,普通高等教育“十三五”规划教材工程图学基础教程习题集(第4版)/,本书嵌入了作者自主研发的可在安卓智能手机上运行的VR 3D 版“工程图学APP” 软件(获2...,普通 高等教育 “ 十三 五 ” 规划 教材 工程图 学 基础教程 习题集 ( 第 4 版 ...,大中专教材教辅
9,美好阅读-落在翅膀上的梦,★著名儿童文学作家、学者王泉根老师倾力主编 ★汇聚众多蜚声海内外的儿童文学名家佳作名篇 ★著...,美好 阅读 - 落 在 翅膀 上 的 梦 ★ 著名 儿童文学 作家 、 学者 王泉根 老师 ...,童书
19,奇幻之旅,本书是《小玛诺林系列》中的第五本，此系列图书曾获1998年西班牙国家青少年文学奖。《小玛诺林...,奇幻 之旅 本书 是 《 小玛诺林 系列 》 中 的 第五 本 ， 此 系列 图书 曾 获 ...,童书
20,云南省地方标准排水降噪沥青路面应用技术指南:DB 53/T 756-2016,本指南规定了排水降噪沥青路面的原材料要求、施工方法、施工质量控制指标以及竣工验收方法等。本指...,云南省 地方 标准 排水 降噪 沥青路面 应用 技术 指南 : DB 53 / T ...,工业技术
22,简明法理学,这是一本简明的法理学教科书，深入浅出是要追求的境界。 本书在形式上有四大特点：其一，知识体系...,简明 法理学 这是 一本 简明 的 法理学 教科书 ， 深入浅出 是 要 追求 的 境界 。...,大中专教材教辅
...,...,...,...,...
212964,中华人民共和国化工行业标准鞋用网眼布HG/T 4808-2015,《中华人民共和国化工行业标准鞋用网眼布HG/T4808-2015》由中华人民共和国工业和信息...,中华人民共和国 化工行业 标准 鞋用 网眼布 HG / T 4808 - 2015 《 ...,工业技术
212967,注塑模具结构设计实战,1、作者从事汽车注塑模具设计十余年，具有丰富的一线设计实践经验。 2、以彩色图解的形式，讲解...,注塑模 具 结构设计 实战 1 、 作者 从事 汽车 注塑 模具设计 十余年 ， 具有 丰富...,工业技术
212981,河北少年儿童出版社少年特战队猎鹰行动/少年特战队10,一个名为秃鹰的恐怖组织在城市中展开暗杀行动，一个个无辜的生命遭遇不幸。为了阻止恐怖分子的猖獗...,河北 少年儿童出版社 少年 特 战队 猎鹰 行动 / 少年 特 战队 10 一个 名为 秃鹰...,童书
212985,经典电动力学-58,"《经典电动力学》从构造洛伦兹变换下不变作用量出发,由最小作用量原理导出基本运动方程。先讲狭义...",经典 电动力学 - 58 《 经典 电动力学 》 从 构造 洛伦兹 变换 下 不变 作用 量...,工业技术


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
import pandas as pd
import re
import jieba
from tqdm import tqdm

# Load the CSV file into a pandas DataFrame with tab (\t) delimiter
train_df = pd.read_csv('train2.csv', sep='\t')
test_df = pd.read_csv('test.csv', sep='\t')

# Load stopwords from the text file
with open('stopwords.txt', 'r') as stopwords_file:
    stopwords = stopwords_file.read().splitlines()

# Merge the "title", "desc", and "text" columns into a single column
train_df['merged'] = train_df['title'] + ' ' + train_df['desc']
test_df['merged'] = test_df['title'] + ' ' + test_df['desc']

# Tokenize using jieba
train_df['merged'] = train_df['merged'].apply(lambda x: ' '.join(jieba.cut(x)))
test_df['merged'] = test_df['merged'].apply(lambda x: ' '.join(jieba.cut(x)))

# Remove stopwords from the merged column
train_df['merged'] = train_df['merged'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))
test_df['merged'] = test_df['merged'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))

# Remove English letters from the merged column
train_df['merged'] = train_df['merged'].apply(lambda x: re.sub(r'[a-zA-Z]+', '', x))
test_df['merged'] = test_df['merged'].apply(lambda x: re.sub(r'[a-zA-Z]+', '', x))

# Data cleaning - remove numbers, letters, punctuation, and tab characters from merged column
train_df['merged'] = train_df['merged'].apply(lambda x: re.sub(r'[\W\d]+', ' ', x))
test_df['merged'] = test_df['merged'].apply(lambda x: re.sub(r'[\W\d]+', ' ', x))

# Remove rows where merged column becomes empty after cleaning
train_df = train_df[train_df['merged'].str.strip().astype(bool)]
test_df = test_df[test_df['merged'].str.strip().astype(bool)]

# 使用合并后的特征列作为特征，标签列作为目标变量
train_book_combined = train_df["merged"].tolist()
labels = train_df["label"].tolist()

# 使用TF-IDF特征提取器
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_book_combined)

# 使用LinearSVM分类器
classifier = LinearSVC()
classifier.fit(X_train, labels)

# 使用合并后的特征列作为特征
test_book_combined = test_df["merged"].tolist()

# 将测试集的合并特征转换为TF-IDF特征
X_test = vectorizer.transform(test_book_combined)

# 预测测试集的标签
y_pred = []

# 使用tqdm显示预测进度条
with tqdm(total=len(test_book_combined), desc='预测进度') as pbar:
    for i in range(len(test_book_combined)):
        pred = classifier.predict(X_test[i])
        y_pred.append(pred[0])
        pbar.update(1)

# 输出预测结果
test_df["预测标签"] = y_pred
print(test_df[["merged", "预测标签"]])

# 提取测试集的实际标签
actual_labels = test_df["label"].tolist()

# 计算准确预测的数量
correct_predictions = sum(1 for pred, actual in zip(y_pred, actual_labels) if pred == actual)

# 计算准确率
accuracy = correct_predictions / len(actual_labels)

print("准确率：", accuracy)


KeyboardInterrupt: 

In [ ]:
import time

start_time = time.time()
# 输出预测结果
test_df["预测标签"] = y_pred
print(test_df[["merged", "预测标签"]])

# 提取测试集的实际标签
actual_labels = test_df["label"].tolist()

# 计算准确预测的数量
correct_predictions = sum(1 for pred, actual in zip(y_pred, actual_labels) if pred == actual)

# 计算准确率
accuracy = correct_predictions / len(actual_labels)

print("准确率：", accuracy)

end_time = time.time()

execution_time = end_time - start_time
print("Execution Time:", execution_time, "seconds")
